In [22]:
import polars as pl
from pathlib import Path
from tcr_format_parsers.common.TriadUtils import (
    FORMAT_MHC_COLS,
    FORMAT_TCR_COLS,
)

iedb_human_II_datpath = Path(
    "/tgen_labs/altin/alphafold3/runs/tcrtrifold-experiments/data/iedb-vdjdb/iedb/human_II/iedb_human_II.csv"
)
iedb_human_II = pl.read_csv(iedb_human_II_datpath)
iedb_human_II_infdir = Path(
    "/tgen_labs/altin/alphafold3/runs/tcrtrifold-experiments/data/iedb-vdjdb/iedb/human_II/inference"
)

unique_mhc_drb = (
    iedb_human_II.group_by(FORMAT_MHC_COLS)
    .agg(
        [pl.col(colname).first() for colname in FORMAT_TCR_COLS]
        + [pl.col("job_name").first()]
    )
    .filter(pl.col("mhc_2_name").str.starts_with("DRB1"))
)

pepseq_all_mhc = pl.read_csv(
    "/home/lwoods/workspace/mdaf3/tmp/all_proteins_PepSeq.csv",
    has_header=False,
    new_columns=["mhc_name", "mhc_seq"],
).with_columns(
    pl.when(pl.col("mhc_name").str.starts_with("D"))
    .then(pl.lit("II"))
    .otherwise(pl.lit("I"))
    .alias("mhc_class")
)
pepseq_infdir = Path()

In [18]:
from mdaf3.FeatureExtraction import (
    raw_helix_indices,
    anneal_helix_indices,
    serial_apply,
)
from mdaf3.AF3OutputParser import AF3Output
from MDAnalysis.analysis.dssp import DSSP


def raw_bsheet_indices(sel):
    # https://docs.mdanalysis.org/2.8.0/documentation_pages/analysis/dssp.html
    bsheet_resindices_boolmask = DSSP(sel).run().results.dssp_ndarray[0, :, 2]
    return sel.residues[bsheet_resindices_boolmask].resindices


def extract_active_site(row, af3_parent_dir):
    af3_output = AF3Output(af3_parent_dir / row["job_name"])

    u = af3_output.get_mda_universe()

    # mhc alpha
    mhc_alpha = u.select_atoms(f"segid B")
    mhc_beta = u.select_atoms(f"segid C")

    if row["mhc_class"] == "I":
        mhc_helices = anneal_helix_indices(raw_helix_indices(mhc_alpha))
        print(mhc_alpha_helices)
    mhc_beta_helices = anneal_helix_indices(raw_helix_indices(mhc_beta))
    print(mhc_beta_helices)

    # not helix specific, just using for convenience
    mhc_alpha_bsheet = anneal_helix_indices(raw_bsheet_indices(mhc_alpha))
    print(mhc_alpha_bsheet)
    mhc_beta_bsheet = anneal_helix_indices(raw_bsheet_indices(mhc_beta))
    print(mhc_beta_bsheet)

    new_row = row.copy()

    return pl.DataFrame(new_row)


serial_apply(
    unique_mhc_drb[0],
    extract_active_site,
    iedb_human_II_infdir,
)

Processing rows: 100%|██████████| 1/1 [00:00<00:00,  4.09it/s]

[[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]]
[[256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290]]
[[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56], [np.int64(98), 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192]]
[[213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 22

mhc_class,mhc_1_chain,mhc_1_species,mhc_1_name,mhc_1_seq,mhc_2_chain,mhc_2_species,mhc_2_name,mhc_2_seq,tcr_1_chain,tcr_1_species,tcr_1_seq,tcr_2_chain,tcr_2_species,tcr_2_seq,job_name
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""II""","""alpha""","""human""","""DRA*01:02""","""KEEHVIIQAEFYLNPDQSGEFMFDFDGDEI…","""beta""","""human""","""DRB1*07:01""","""GDTQPRFLWQGKYKCHFFNGTERVQFLERL…","""alpha""","""human""","""GQQVMQIPQYQHVQEGEDFTTYCNSSTTLS…","""beta""","""human""","""NAGVTQTPKFRILKIGQSMTLQCTQDMNHN…","""46327371a8c4a2a0a5556ba17838de…"


In [19]:
pepseq_all_mhc

NameError: name 'pepseq_all_mhc' is not defined